In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ventilator-pressure-prediction/sample_submission.csv
/kaggle/input/ventilator-pressure-prediction/train.csv
/kaggle/input/ventilator-pressure-prediction/test.csv
/kaggle/input/hpc-files/ventilator_extraFeatures_ventillator_RNN_rerun_20.csv


In [2]:
##Analyze the ventillator pressure dataset
import pandas as pd
import numpy as np
##Create a NN model
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Activation, Dropout, LSTM, GRU, Bidirectional
from tensorflow.keras.callbacks import CSVLogger, TensorBoard, ModelCheckpoint, EarlyStopping
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras.preprocessing import timeseries_dataset_from_array
import gc


trainDataPath = "/kaggle/input/ventilator-pressure-prediction/train.csv"
df_data = pd.read_csv(trainDataPath, header=0)

df_data.head()
#df_data.describe()

# testDataPath = "/kaggle/input/ventilator-pressure-prediction/test.csv"
# df_test = pd.read_csv(testDataPath, header=0)

# df_test.head()

DEBUG= False
if DEBUG:
    df_data = df_data.iloc[:80*100, :]
#     df_test = df_test.iloc[:80*100, :]
print(df_data.shape)
# print(df_test.shape)

(6036000, 8)


In [3]:
def searchNearest(prediction):
    idx = np.searchsorted(df_data_y_sort, prediction)
    #print("prediction= %s, idx= %s" %(prediction,  idx))
    if (idx >= df_data_y_sort.shape[0]-1):
        ## If the number is greater than the largest value in sort array, return the largest element
        return df_data_y_sort[-1]
    
    lowerVal = df_data_y_sort[idx]
    upperVal = df_data_y_sort[idx+1]
    return lowerVal if np.abs(lowerVal - prediction) < np.abs(upperVal - prediction) else upperVal


In [4]:
df_y_data = df_data["pressure"].copy()
targets = df_y_data.to_numpy().reshape(-1, 80)
df_data_y_sort = np.sort(df_y_data.unique())
#print(df_data_y_sort)

In [5]:
subFile = "/kaggle/input/hpc-files/ventilator_extraFeatures_ventillator_RNN_rerun_20.csv"
oldName = subFile.split("/")[-1].split("_")
newName = "./" + "_".join(oldName[:-1]) + "_round" + oldName[-1]
test_sub = pd.read_csv(subFile, header=0)
#test_sub.shape
test_sub["pressure"] = test_sub["pressure"].apply(searchNearest)
print(test_sub.head())
test_sub.to_csv(newName, index=False)

   id  pressure
0   1  6.259305
1   2  5.907794
2   3  7.243535
3   4  7.876254
4   5  9.141693
